# 1. Setup

## 1.1 Module import

Import necessary modules

In [11]:
import tensorflow as tf
import keras
import dataset_utils

## 1.2 Dataset import using opencv

Define parameters for the image import

In [12]:
img_width = 100
img_height = 40

train_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped'
validation_dataset_path = r'/mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped'

Import the data for the training dataset

In [3]:
train_labels, train_images = dataset_utils.load_dataset(path_to_dataset = train_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 18777 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 18777 images and 18777 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/training_dataset/all_cropped
Loaded images have shape (40, 100, 3).


Import the data for the validation dataset

In [4]:
validation_labels, validation_images = dataset_utils.load_dataset(path_to_dataset = validation_dataset_path, change_contrast = True, grayscale = True, image_size = (img_width, img_height))

Loading 1812 images and labels. This might take a while, please wait...
Progress 100 %
Loaded 1812 images and 1812 labels from directory /mnt/d/Dev/DHBW/DHBW_Studienarbeit/training_images/validation_dataset/all_cropped
Loaded images have shape (40, 100, 3).


# 2. Transfer learning

## 2.1 Construct the new model

Load the base model

In [13]:
IMG_SIZE = (img_height, img_width)
IMG_SHAPE = IMG_SIZE + (3,)

base_model_v3 = tf.keras.applications.MobileNetV3Small(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_v3.trainable = False

base_model_v2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_v2.trainable = False

base_model_v1 = tf.keras.applications.MobileNet(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model_v1.trainable = False



Add the custom classification layer to it

In [14]:
inputs = keras.Input(shape = (img_height, img_width, 3))

x = base_model_v3(inputs, training = False)
x = tf.keras.layers.Flatten()(x)
outputs_3 = tf.keras.layers.Dense(3, activation = 'softmax')(x)

y = base_model_v2(inputs, training=False)
y = tf.keras.layers.Flatten()(y)
outputs_2 = tf.keras.layers.Dense(3, activation = 'softmax')(y)

z = base_model_v1(inputs, training=False)
z = tf.keras.layers.Flatten()(z)
outputs_1 = tf.keras.layers.Dense(3, activation = 'softmax')(z)

model_v1 = keras.Model(inputs, outputs_1)
model_v2 = keras.Model(inputs, outputs_2)
model_v3 = keras.Model(inputs, outputs_3)

model_v3.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 40, 100, 3)]      0         
                                                                 
 MobilenetV3small (Function  (None, 2, 4, 576)         939120    
 al)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 4608)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 13827     
                                                                 
Total params: 952947 (3.64 MB)
Trainable params: 13827 (54.01 KB)
Non-trainable params: 939120 (3.58 MB)
_________________________________________________________________


## 2.2 Start the initial learning process

Define Parameters for the learning process

In [6]:
base_learning_rate = 0.03
initial_epochs = 10

Compile the before built model

In [7]:
model_v1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model_v2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model_v3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

Train the model!

In [ ]:
def scheduler(epoch, lr):
    if epoch == 0:
        return lr
    elif epoch % 5 == 0:
        return lr * 0.9 
    else: 
        return lr

In [ ]:
#callback = keras.callbacks.LearningRateScheduler(scheduler)
callback = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator()
batch_size = 32

In [ ]:


history_v1 = model_v1.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs)


In [ ]:
history_v2 = model_v2.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs)

In [10]:
history_v3 = model_v3.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=(validation_images, validation_labels), epochs=initial_epochs)

Epoch 1/10


2024-05-23 04:44:34.796788: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


  1/586 [..............................] - ETA: 40:16 - loss: 2.3177 - accuracy: 0.1562

2024-05-23 04:44:35.633582: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f08d11d1cd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-23 04:44:35.633635: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 4GB Laptop GPU, Compute Capability 8.6
2024-05-23 04:44:35.639990: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1716432275.713578  332638 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


586/586 [==============================] - 16s 20ms/step - loss: 2.1908 - accuracy: 0.8852 - val_loss: 0.7557 - val_accuracy: 0.9465
Epoch 2/10
586/586 [==============================] - 11s 19ms/step - loss: 1.4381 - accuracy: 0.9309 - val_loss: 1.2302 - val_accuracy: 0.9514
Epoch 3/10
586/586 [==============================] - 11s 19ms/step - loss: 1.2747 - accuracy: 0.9438 - val_loss: 3.6912 - val_accuracy: 0.9100
Epoch 4/10
586/586 [==============================] - 11s 18ms/step - loss: 1.5123 - accuracy: 0.9432 - val_loss: 1.3488 - val_accuracy: 0.9614
Epoch 5/10
586/586 [==============================] - 11s 19ms/step - loss: 1.5022 - accuracy: 0.9501 - val_loss: 1.4819 - val_accuracy: 0.9614
Epoch 6/10
586/586 [==============================] - 11s 19ms/step - loss: 1.1780 - accuracy: 0.9586 - val_loss: 4.4527 - val_accuracy: 0.8951
Epoch 7/10
586/586 [==============================] - 11s 19ms/step - loss: 1.3992 - accuracy: 0.9549 - val_loss: 4.0123 - val_accuracy: 0.9200
Epo

: 

In [ ]:
model.evaluate(x = validation_images, y = validation_labels)

# 3. Fine tuning the base model

Freeze all layers except for the uppermost layers of the base model. Layzers will be frozen from ***fine_tune_from*** to ***len(base_model.layers)***.

In [ ]:
base_model.trainable = True
fine_tune_from = len(base_model.layers) - 25

for layer in base_model.layers[:fine_tune_from]:
    layer.trainable = False

# don't train the dense layer
model.layers[len(model.layers) - 1].trainable = False
model.layers[len(model.layers) - 2].trainable = False

model.summary()

Compile the model again but decrease the learning rate in order to avoid overfitting.

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/100), loss='categorical_crossentropy', metrics=['accuracy'])

Train the uppermost layers of the base model and proceed where you left off in the initial training by using the last epoch of the first training as the initial epoch for fine tuning.

In [ ]:
finetune_history = model.fit(generator.flow(train_images, train_labels, batch_size), steps_per_epoch=len(train_images)/batch_size, validation_data=generator.flow(validation_images, validation_labels, batch_size), initial_epoch=history.epoch[-1], epochs=initial_epochs + 10, callbacks=callback)

In [ ]:
print(finetune_history.history.keys())
key = 'accuracy'
for loss in history.history[key]:
    print(loss, end='\r\n')
for loss in finetune_history.history[key]:
    print(loss, end='\r\n')

# 4. Save the model

Info: Loading can be done via: ***model = tf.keras.models.load_model(PATH_TO_MODEL_FILE)***


In [10]:
# for now until it is implemented properly: manually make sure that the path exists (in this case '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/')
save_path = '/mnt/d/Dev/DHBW/DHBW_Studienarbeit/models/model_v3.keras'

model_v3.save(save_path)